# Use Spark to predict business area for car rental company with `ibm-watson-machine-learning`

This notebook contains steps and code to create a predictive model, and deploy it on WML. This notebook introduces commands for pipeline creation, model training, model persistance to Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.6 and Apache® Spark 2.4.

You will use **car_rental_training** dataset.


## Learning goals

The learning goals of this notebook are:

-  Load a CSV file into an Apache® Spark DataFrame.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create an Apache® Spark machine learning pipeline.
-  Train and evaluate a model.
-  Persist a pipeline and model in Watson Machine Learning repository.
-  Deploy a model for online scoring using Wastson Machine Learning API.
-  Score sample scoring data using the Watson Machine Learning API.


## Contents

This notebook contains the following parts:
1. [Setup](#setup)
2. [Load and explore data](#load)
3. [Create an Apache Spark machine learning model](#model)
4. [Store the model in the Watson Machine Learning repository](#persistence)
5. [Deploy the model in the IBM Cloud](#deployment)
6. [Score](#score)
7. [Clean up](#cleanup)
8. [Summary and next steps](#summary)

**Note:** This notebook works correctly with kernel `Python 3.6 with Spark 2.4`, please **do not change kernel**.

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">here</a>).

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

In result, get the value of `api_key` from the output.


Location of your WML instance can be retrieved in the following way:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance WML_INSTANCE_NAME
```

In result, get the value of `location` from the output.

**Tip**: Your `Cloud API key` can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below. You can also get a service specific url by going to the [**Endpoint URLs** section of the Watson Machine Learning docs](https://cloud.ibm.com/apidocs/machine-learning).  You can check your instance location in your  <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance details.

You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.

**Action**: Enter your `api_key` and `location` in the following cell.

In [ ]:
api_key = 'PASTE YOUR PLATFORM API KEY HERE'
location = 'PASTE YOUR INSTANCE LOCATION HERE'

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install ibm-watson-machine-learning

In [ ]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create one.

- Click New Deployment Space
- Create an empty space
- Select Cloud Object Storage
- Select Watson Machine Learning instance and press Create
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [ ]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [ ]:
client.set.default_space(space_id)

### Test Spark

In [ ]:
try:
    from pyspark.sql import SparkSession
except:
    print('Error: Spark runtime is missing. If you are using Watson Studio change the notebook runtime to Spark.')
    raise

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache Spark DataFrame and perform a basic exploration.

Read data into Spark DataFrame from file.

In [ ]:
import os
from wget import download

sample_dir = 'spark_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
filename = os.path.join(sample_dir, 'car_rental_training_data.csv')
if not os.path.isfile(filename):
    filename = download('https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/data/cars-4-you/car_rental_training_data.csv', out=sample_dir)

In [ ]:
spark = SparkSession.builder.getOrCreate()

df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .option("delimiter", ";")\
  .load(filename)
df_data.take(3)

### Explore data

In [ ]:
df_data.printSchema()

As you can see, the data contains eleven fields. `Business_Area` field is the one you would like to predict using feedback data in `Customer_Service` field.

In [ ]:
print("Number of records: " + str(df_data.count()))

Let's see distribution of target field.

In [ ]:
df_data.select('Business_Area').groupBy('Business_Area').count().show(truncate=False)

<a id="model"></a>
## 3. Create an Apache Spark machine learning model

In this section you will learn how to:

- [3.1 Prepare data for model training and evaluation](#prep)
- [3.2 Create an Apache Spark machine learning pipeline](#pipe)
- [3.3 Train a model](#train)

<a id="prep"></a>
### 3.1 Prepare data for model training and evaluation

In this subsection you will split your data into: train and test data set.

In [ ]:
train_data, test_data = df_data.select("ID", "Customer_Service", "Business_Area").randomSplit([0.8, 0.2], 24)

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))

### 3.2 Create the pipeline<a id="pipe"></a>

In this section you will create an Apache Spark machine learning pipeline and then train the model.

In [ ]:
from pyspark.ml.feature import StringIndexer, IndexToString, HashingTF, IDF, Tokenizer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model
from pyspark.sql.types import *

In the first data preprocessing step, create features from `Customer_Service` field.

In [ ]:
tokenizer = Tokenizer(inputCol="Customer_Service", outputCol="words")
hashing_tf = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol='hash')
idf = IDF(inputCol=hashing_tf.getOutputCol(), outputCol="features", minDocFreq=5)

In the following step, use the StringIndexer transformer to convert `Business_Area` to numeric.

In [ ]:
string_indexer_label = StringIndexer(inputCol="Business_Area", outputCol="label").fit(train_data)

Add decision tree model to predict `Business_Area`.

In [ ]:
dt_area = DecisionTreeClassifier(labelCol="label", featuresCol=idf.getOutputCol())

Finally, setup transformer to convert the indexed labels back to original labels.

In [ ]:
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=string_indexer_label.labels)

In [ ]:
pipeline = Pipeline(stages=[tokenizer, hashing_tf, idf, string_indexer_label, dt_area, label_converter])

### 3.3 Train the model<a id="train"></a>

In this subsection you will train model and evaluate its accuracy.

In [ ]:
model = pipeline.fit(train_data)

In [ ]:
predictions = model.transform(test_data)
predictions.select('Customer_Service','Business_Area','predictedLabel').show(3)

In [ ]:
predictions.printSchema()

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("Accuracy = %3.2f" % accuracy)

**Note:** Accuracy of the model is low, however based on customer comment more than one Business Area could be selected. In such cases top k (for example k=3) would be more suited for model evaluation.

<a id="persistence"></a>
## 4. Persist model

In this section you will learn how to store your pipeline and model in Watson Machine Learning repository by using python client libraries.

**Note**: Apache® Spark 2.4 is required.

#### Save training data in your Cloud Object Storage

ibm-cos-sdk library allows Python developers to manage Cloud Object Storage (COS).

In [ ]:
import ibm_boto3
from ibm_botocore.client import Config

**Action**: Put credentials from Object Storage Service in Bluemix here.

In [ ]:
cos_credentials = {
                  "apikey": "***",
                  "cos_hmac_keys": {
                    "access_key_id": "***",
                    "secret_access_key": "***"
                  },
                  "endpoints": "***",
                  "iam_apikey_description": "***",
                  "iam_apikey_name": "***",
                  "iam_role_crn": "***",
                  "iam_serviceid_crn": "***",
                  "resource_instance_id": "***"
                }

In [ ]:
connection_apikey = cos_credentials['apikey']
connection_resource_instance_id = cos_credentials["resource_instance_id"]
connection_access_key_id = cos_credentials['cos_hmac_keys']['access_key_id']
connection_secret_access_key = cos_credentials['cos_hmac_keys']['secret_access_key']

**Action**: Define the service endpoint we will use. <br>
**Tip**: You can find this information in Endpoints section of your Cloud Object Storage intance's dashbord.

In [ ]:
service_endpoint = 'https://s3.us.cloud-object-storage.appdomain.cloud'

You also need IBM Cloud authorization endpoint to be able to create COS resource object.

In [ ]:
auth_endpoint = 'https://iam.cloud.ibm.com/identity/token'

We create COS resource to be able to write data to Cloud Object Storage.

In [ ]:
cos = ibm_boto3.resource('s3',
                         ibm_api_key_id=cos_credentials['apikey'],
                         ibm_service_instance_id=cos_credentials['resource_instance_id'],
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

Now you will create bucket in COS and copy `training dataset` for model from **car_rental_training_data.csv**.

In [ ]:
from uuid import uuid4

bucket_uid = str(uuid4())

score_filename = "car_rental_training_data.csv"
buckets = ["car-rental-" + bucket_uid]

In [ ]:
for bucket in buckets:
    if not cos.Bucket(bucket) in cos.buckets.all():
        print('Creating bucket "{}"...'.format(bucket))
        try:
            cos.create_bucket(Bucket=bucket)
        except ibm_boto3.exceptions.ibm_botocore.client.ClientError as e:
            print('Error: {}.'.format(e.response['Error']['Message']))

In [ ]:
bucket_obj = cos.Bucket(buckets[0])

print('Uploading data {}...'.format(score_filename))
with open(filename, 'rb') as f:
    bucket_obj.upload_fileobj(f, score_filename)
print('{} is uploaded.'.format(score_filename))

### 4.2 Save the pipeline and model<a id="save"></a>

In [ ]:
training_data_references = [
                {
                    "id":"car-rental-training",
                    "type": "s3",
                    "connection": {
                        "access_key_id": connection_access_key_id,
                        "endpoint_url": service_endpoint,
                        "secret_access_key": connection_secret_access_key
                    },
                    "location": {
                        "bucket": buckets[0],
                        "path": score_filename,
                    }
                }
            ]

In [ ]:
saved_model = client.repository.store_model(
    model=model, 
    meta_props={
        client.repository.ModelMetaNames.NAME:"CARS4U - Business Area Prediction Modeljj",
        client.repository.ModelMetaNames.SPACE_UID: space_id,
        client.repository.ModelMetaNames.TYPE: "mllib_2.4",
        client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: client.software_specifications.get_id_by_name('spark-mllib_2.4'),
        client.repository.ModelMetaNames.TRAINING_DATA_REFERENCES: training_data_references,
        client.repository.ModelMetaNames.LABEL_FIELD: "Business_Area",
    },  
    training_data=train_data, 
    pipeline=pipeline)

Get saved model metadata from Watson Machine Learning.

In [ ]:
published_model_id = client.repository.get_model_uid(saved_model)

print("Model Id: " + str(published_model_id))

**Model Id** can be used to retrive latest model version from Watson Machine Learning instance.

Below you can see stored model details.

In [ ]:
client.repository.get_model_details(published_model_id)

<a id="deploy"></a>
## 5. Deploy model in the IBM Cloud

In this section you will learn how to create model deployment in the IBM Cloud and retreive information about scoring endpoint.

In [ ]:
deployment_details = client.deployments.create(
    published_model_id, 
    meta_props={
        client.deployments.ConfigurationMetaNames.NAME: "CARS4U - Business Area Prediction Model deployment",
        client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
)

In [ ]:
deployment_details

<a id="score"></a>
## 6. Score

In [ ]:
fields = ['ID', 'Gender', 'Status', 'Children', 'Age', 'Customer_Status','Car_Owner', 'Customer_Service', 'Business_Area', 'Satisfaction']
values = [3785, 'Male', 'S', 1, 17, 'Inactive', 'Yes', 'The car should have been brought to us instead of us trying to find it in the lot.', 'Product: Information', 0]

In [ ]:
import json

payload_scoring = {"input_data": [{"fields": fields,"values": [values]}]}
scoring_response = client.deployments.score(client.deployments.get_id(deployment_details), payload_scoring)

print(json.dumps(scoring_response, indent=3))

<a id="cleanup"></a>
## 7. Clean up

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 8. Summary and next steps     

 You successfully completed this notebook! You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. Check out our [Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics) for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Amadeusz Masny**, Python Software Developer in Watson Machine Learning at IBM

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.